In [ ]:
!pip install transformers torch datasets accelerate

import torch
from datasets import load_dataset
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments


In [ ]:
# Verify GPU
print("GPU Available:", torch.cuda.is_available())
print("GPU Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

GPU Available: True
GPU Device: NVIDIA A100-SXM4-40GB


In [ ]:
# Load XSum dataset
dataset = load_dataset("xsum")
train_data = dataset["train"]
validation_data = dataset["validation"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.24k [00:00<?, ?B/s]

xsum.py:   0%|          | 0.00/5.76k [00:00<?, ?B/s]

The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


(…)SUM-EMNLP18-Summary-Data-Original.tar.gz:   0%|          | 0.00/255M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [ ]:
# Load pre-trained BART model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [ ]:

# Define preprocessing function
def preprocess_function(batch):
    inputs = tokenizer(
        batch["document"],
        max_length=1024,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    targets = tokenizer(
        batch["summary"],
        max_length=80,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    batch["input_ids"] = inputs["input_ids"]
    batch["attention_mask"] = inputs["attention_mask"]
    batch["labels"] = targets["input_ids"]
    return batch

In [ ]:
# Apply preprocessing with batching for speed
tokenized_train = train_data.map(
    preprocess_function,
    batched=True,
    remove_columns=train_data.column_names,
    num_proc=2  # Use multiple processes for faster preprocessing
)
tokenized_val = validation_data.map(
    preprocess_function,
    batched=True,
    remove_columns=validation_data.column_names,
    num_proc=2
)

Map (num_proc=2):   0%|          | 0/204045 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/11332 [00:00<?, ? examples/s]

In [ ]:
# Define optimized training arguments
# training_args = TrainingArguments(
#     output_dir="./xsum_bart",
#     evaluation_strategy="steps",
#     eval_steps=500,  # Evaluate every 500 steps
#     save_strategy="steps",
#     save_steps=500,  # Save every 500 steps
#     learning_rate=2e-5,
#     per_device_train_batch_size=64,  # Increase if your GPU memory allows
#     per_device_eval_batch_size=4,
#     num_train_epochs=1,
#     weight_decay=0.01,
#     save_total_limit=3,
#     gradient_accumulation_steps=4,  # Accumulate gradients to simulate larger batch size
#     fp16=True,  # Use mixed precision training
#     report_to="tensorboard",  # Monitor training with tensorboard
#     logging_steps=100,
#     load_best_model_at_end=True,
#     metric_for_best_model="rouge1",  # or any other metric you're using

# )

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_train,
#     eval_dataset=tokenized_val,
# )

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
#trainer.train()

In [ ]:
# # Save in safetensors format (more efficient)
# model.save_pretrained(output_dir, safe_serialization=True)

# # Or save with compression
# model.save_pretrained(output_dir, use_torch_save=True, max_shard_size="500MB")

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Bart_model/Train",
    num_train_epochs=1,
    per_device_train_batch_size=32,  # Increased from 4 to 8
    per_device_eval_batch_size=32,   # Increased from 4 to 8
    learning_rate=5e-5,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=50,
    logging_steps=10,
    fp16=True,
    report_to="none",
    # Add gradient accumulation for effective larger batch size
    gradient_accumulation_steps=2,  # This makes effective batch size = 8 * 2 = 16
    save_strategy="steps",
    save_steps=50,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",  # Save based on evaluation loss
    greater_is_better=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

# Train
trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at ../aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Step,Training Loss,Validation Loss
50,0.304000,0.799876
100,0.656100,0.657467
150,0.634400,0.649587
200,0.613100,0.630663
250,0.611300,0.640959
300,0.609000,0.632632
350,0.606600,0.632933
400,0.616900,0.637336
450,0.630900,0.634288
500,0.604900,0.625865


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at ../aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig f

KeyboardInterrupt: 

In [ ]:
final_output_dir = "/content/drive/MyDrive/Bart_model/final"
model.save_pretrained(final_output_dir)
tokenizer.save_pretrained(final_output_dir)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('/content/drive/MyDrive/Bart_model/final/tokenizer_config.json',
 '/content/drive/MyDrive/Bart_model/final/special_tokens_map.json',
 '/content/drive/MyDrive/Bart_model/final/vocab.json',
 '/content/drive/MyDrive/Bart_model/final/merges.txt',
 '/content/drive/MyDrive/Bart_model/final/added_tokens.json')

In [ ]:

# Save the quick-trained model
model.save_pretrained("./bart_trained_secondtime_longer")
tokenizer.save_pretrained("./bart_trained_secondtime_longer")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./bart_trained_secondtime_longer/tokenizer_config.json',
 './bart_trained_secondtime_longer/special_tokens_map.json',
 './bart_trained_secondtime_longer/vocab.json',
 './bart_trained_secondtime_longer/merges.txt',
 './bart_trained_secondtime_longer/added_tokens.json')